In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp utils

# Objective 2 : Predict Probability of Failure in Next N days


Predictive Maintenance (PdM) is a great application of Survival Analysis since it consists in predicting when equipment failure will occur and therefore alerting the maintenance team to prevent that failure.

### ` Objectives`
> - To Predict Probability of Failure in Next N days

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import shap
from boruta import BorutaPy

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import QuantileTransformer
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM

from PredictiveMaintenance2 import Datasets,Visualize,FeatureEng,Model

F:\Lalitha\anaconda3\envs\Predictive-Maintenance\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature selection

### load dataset

In [ ]:
# load pre processed dataset
machineData = pd.read_csv('Machine_Data_Preprocessed.csv')
machineData.head(2)

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,RUL,SurvivalTime
0,2015-01-01,0,0,141503600,0,0,1,19,494462,16,16,3,18,1
1,2015-01-01,1,0,55587136,0,0,0,7,199132,0,0,0,214,1


In [ ]:
# sort values by device and date
machineData = machineData.sort_values(['device','date'],ascending= True).reset_index(drop=True)

# get last record of each device
machineData['date'] = pd.to_datetime(machineData['date'])
last_date = machineData.groupby('device')['date'].transform(max) == machineData['date']

In [ ]:
# Filter the data to keep only the rows with the maximum observation date for each device
machineData = machineData[last_date].reset_index(drop=True)

# Print the final output
print(machineData.head(2))

        date  device  failure    metric1  metric2  metric3  metric4  metric5   
0 2015-01-19       0        1   64499464        0        0        1       19  \
1 2015-08-03       1        1  110199904      240        0        0        8   

   metric6  metric7  metric8  metric9  RUL  SurvivalTime  
0   514661       16       16        3    0            19  
1   294852        0        0        0    0           215  


### split the data

In [ ]:
x = machineData.drop(['date','failure','RUL','SurvivalTime'],axis=1)

# Create a structured array for the survival analysis
y = np.zeros(len(machineData), dtype=[('failure', 'bool'), ('SurvivalTime', 'float')])
y['failure'] = machineData['failure'].astype(bool)
y['SurvivalTime'] = machineData['SurvivalTime'].astype(float)


In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [ ]:
# Normalize the data using quantile normalization
qn = QuantileTransformer(output_distribution='normal', random_state=42)
x_train_norm = qn.fit_transform(x_train)
x_test_norm = qn.transform(x_test)

n_quantiles (1000) is greater than the total number of samples (116). n_quantiles is set to n_samples.


### SelectFromModel

# Modeling - CoxPH Survival Model

In [ ]:
# Create a instance Cox PH survival model
estimator = CoxPHSurvivalAnalysis(alpha=0.05)

# Fit the model to the training data
estimator.fit(x_train_norm, y_train)

CoxPHSurvivalAnalysis(alpha=0.05)

# Predictions


In [ ]:
# Predict the probability of failure in the next n days for each machine in the testing set
y_pred = estimator.predict(x_test_norm)

In [ ]:
# Predict the survival functions for the testing set
survival_functions = estimator.predict_survival_function(x_test_norm)

In [ ]:
print(len(survival_functions))

21


Calculate the probability of failure in the next n days using the predicted hazard ratios and the survival function

In [ ]:
# Get the probability of failure in the next n days
n_days = 20 
failure_prob = []

for sf in survival_functions:
    t_idx = np.argmin(np.abs(sf.x - n_days))
    failure_prob.append(1 - sf.y[t_idx])
    
failure_prob = np.array(failure_prob)

# Print the prObability of failure for each device in the testing set
for i, prob in enumerate(failure_prob):
    if prob>0.5:
        print(f"----Device-{i} : {prob*100:.2f}-----")
    else:
        print(f"Device-{i} : {prob*100:.2f}") 

Device-0 : 19.05
Device-1 : 31.42
Device-2 : 2.98
Device-3 : 4.25
Device-4 : 17.55
Device-5 : 3.52
Device-6 : 13.92
Device-7 : 7.97
Device-8 : 39.02
Device-9 : 18.59
----Device-10 : 53.53-----
Device-11 : 5.26
Device-12 : 8.73
Device-13 : 5.06
Device-14 : 4.01
Device-15 : 9.44
Device-16 : 18.29
Device-17 : 44.52
Device-18 : 8.25
Device-19 : 5.32
Device-20 : 7.35


# Validations

In [ ]:
# Evaluate the performance of the model on 
c_index = concordance_index_censored(y_test['failure'], y_test['SurvivalTime'], y_pred)
print(f"C-index: {c_index}")

C-index: (0.5738636363636364, 101, 75, 0, 0)


# Modeling - kaplan-Meier survival function from lifelines

In [ ]:
#| hide
"""
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times

# Prepare the data for Kaplan-Meier model
train_frac = 0.7
train_size = int(train_frac * len(machineData))
train_data = machineData[:train_size]
test_data = machineData[train_size:]

kmf = KaplanMeierFitter()
kmf.fit(train_data['SurvivalTime'],train_data['failure'])

n_days = 10
# Predict the survival function using Kaplan-Meier model
sf = kmf.survival_function_at_times(np.array([n_days]))
print(sf)

# Get the probability of failure in next n days for each device
prob_failure = []

for i, row in test_data.iterrows():
    if row['SurvivalTime'] <= n_days:
        prob_failure.append(1.0)
    else:
        median_time = median_survival_times(kmf.predict(row['SurvivalTime']))
        prob_failure.append(1 - sf[median_time])

# Compute the concordance index to evaluate the model performance
c_index = concordance_index_censored(test_data['failure'], test_data['SurvivalTime'], prob_failure)
print("Concordance index:", c_index)

"""

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()